In [1]:
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from keras.models import Sequential
from keras.utils import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np### 데이터 준비 완료 -----------

from sklearn.metrics import accuracy_score

In [2]:
path = r"C:\\Users\\USER\\Desktop\\font_original"           #원본 이미지 경로
copy_path = r"C:\\Users\\USER\\Desktop\\font_classifier"    #train/test/val 폴더로 분류한 이미지 경로

In [3]:
#train/test 폴더로 분류
import splitfolders

splitfolders.ratio(path, output=copy_path, seed = 43, ratio = (.7,.3))

Copying files: 55860 files [02:27, 378.82 files/s]


In [5]:
trainFileList = os.listdir(copy_path+'\\train')                    #train 폴더 경로
testFileList = os.listdir(copy_path+'\\test')                       #val 폴더 경로

In [6]:
trainFileList = [x for x in trainFileList if 'jpg' in x]
testFileList = [x for x in testFileList if 'jpg' in x]

In [7]:
label_list = list(set(map(lambda x : x[-6:-4], trainFileList))) 

In [8]:
trainDataGen = ImageDataGenerator(rescale=1./255)

In [10]:
trainGenSet = trainDataGen.flow_from_directory(
    copy_path + '\\train',
    batch_size=64,
    target_size=(28,28),
    class_mode='categorical'
)

Found 39100 images belonging to 5 classes.


In [11]:
testDataGen = ImageDataGenerator(rescale=1./255)

In [18]:
testGenSet = testDataGen.flow_from_directory(
    copy_path + '\\test',
    target_size=(28,28),
    batch_size=64,
    class_mode='categorical'
)

Found 16760 images belonging to 5 classes.


In [20]:
valDataGen = ImageDataGenerator(rescale=1./255,
                                 rotation_range = 15,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.1,
                                 horizontal_flip=False,
                                 vertical_flip=False,
                                 fill_mode='nearest')

valGenSet = valDataGen.flow_from_directory(
    copy_path + '\\train',
    target_size=(28,28),
    batch_size=64,
    class_mode='categorical'
)

Found 39100 images belonging to 5 classes.


In [21]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), padding='same', input_shape=(28,28,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(18, activation='softmax'))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 7, 7, 128)        

In [23]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [25]:
model.fit(
    trainGenSet,
    steps_per_epoch=40,
    epochs=400,
    validation_data=valGenSet,
    validation_steps=20,
)

Epoch 1/400


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "C:\Users\USER\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\USER\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\USER\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\USER\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\USER\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\USER\anaconda3\lib\asyncio\base_events.py", line 601, in run_fore